In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [10]:
fname = "property_years_4_19_24.geojson"

df_gpd = gpd.read_file(fname)


In [11]:
df_pd = pd.DataFrame(df_gpd)

In [17]:
bins = [1890,1930,1960,1990,2025]
labels = ["1890-1930", "1930-1960", "1960-1990","1990-2025"]

df_pd['era'] = pd.cut(x=df_pd['year_start'], bins=bins, labels=labels)

df_pd['era'] = df_pd['era'].astype(str)

In [18]:
with_eras = gpd.GeoDataFrame(df_pd, crs="EPSG:4326")


In [19]:
with_eras.to_file("with_era.geojson", driver="GeoJSON")  

In [5]:
import requests
import json

url = 'https://api.maptiler.com/maps/dataviz/style.json?key=aXEOZFRxlPWOaJqsWprN'
r = requests.get(url)
tiles = r.json()    

with open("tiles.json", "w") as outfile: 
    json.dump(tiles, outfile)


AttributeError: 'dict' object has no attribute 'to_json'